# 概述

- 常用rtmp、m3u8、flv、mp4直播流在线测试地址    https://blog.csdn.net/u014696856/article/details/135365211
- 在 OpenCV 中优化 RTSP 视频处理：克服 FPS 差异和缓冲问题  https://www.nxrte.com/jishu/47826.html

倍速播放需要在相同的时间内播放更多的帧，意味着减少每帧之间的显示时间，而不是减少解码的帧数。但跳帧是减少解码和处理的帧数。

cv2.VideoCapture.read() 包含图像的解码过程，速度较慢。可以用 cv2.VideoCapture.grab() + cv2.VideoCapture.retrieve() 来替代

## 流媒体播放器实现视频流的倍速播放

该过程主要涉及两个方面：视频流和音频流的处理。

视频流倍速播放：
1. 改变播放帧率：视频播放的基本原理是按帧序列显示图像，改变播放帧率可以实现倍速播放。如果原始视频的帧率是30fps，以200%的速度播放，相当于将帧率提升到60fps，即每帧的时间间距缩短，从而实现快速播放。
2. 修改时间戳（PTS）：通过调整视频流中每帧的显示时间戳（PTS），可以改变视频的播放速度。例如，使用ffmpeg的  setpts  过滤器，可以通过修改PTS值来加快或减慢视频播放速度。
3. 跳帧：在不改变帧率的情况下，通过跳过某些帧来实现倍速播放。例如，如果视频一秒30帧，加速一倍可以通过播放1、3、5...等奇数帧来实现。

音频流倍速播放：
1. 音频重采样：音频的倍速播放涉及到音频信号的时域拉伸或压缩。一种简单的方法是均匀地扔掉一部分音频数据，或者通过算法对音频信号进行处理，以达到变速不变调的效果。例如，44100Hz采样率的音频，如果要加速为200%播放，可以通过重采样滤波，将采样点减少一半，然后以原来的采样率播放，实现音频的倍速播放。
2. 时域压扩算法（TSM算法）：一些播放器使用TSM算法来实现音频的倍速播放。这种方法通过对输入的语音信号进行不断的分帧与合帧处理，最终合成新的信号以达到倍速的效果。例如，Sonic库和SoundTouch库都使用了基于TSM算法的方法，但它们在处理音频信号合成时采用不同的技术。
3. 调整音频码率和振幅：在某些播放器如ExoPlayer中，倍速播放通过调整音频码率和振幅实现，利用  AudioTrack  组件的  setPlaybackRate  方法来调整音频播放速率，确保与视频流保持同步。音视频同步在实现倍速播放时，保持音视频同步是非常重要的。播放器需要确保在改变播放速度时，音频和视频的相对时间关系保持一致，避免出现音视频不同步的问题。

音视频同步在实现倍速播放时，保持音视频同步是非常重要的。播放器需要确保在改变播放速度时，音频和视频的相对时间关系保持一致，避免出现音视频不同步的问题。

总结来说，流媒体播放器实现视频流的倍速播放主要通过改变视频帧率、调整时间戳、跳帧等方法对视频流进行处理，而音频流则通过音频重采样、时域压扩算法等技术实现变速播放，同时确保音视频同步。

## 流媒体播放器实现跳帧

1. 调整播放帧率：播放器可以通过调整视频的播放帧率来实现跳帧。例如，如果视频原始帧率是30fps，播放器可以设置为60fps播放，这样就会跳过一半的帧，实现快速播放的效果。这种方法通过修改视频滤镜（Video Filter）来实现，例如在MPV播放器中可以使用命令行选项  --vf=fps=60/2  来将视频帧率降低一半，即跳过一半的帧进行播放。
2. 修改时间戳（PTS）：播放器可以修改视频流中每帧的显示时间戳（PTS），通过  setpts  过滤器来控制播放速度的倍数。例如，  setpts=0.5*PTS 表示加快2倍播放速度，而  setpts=2*PTS  表示减慢到0.5倍播放速度。
3. 关键帧跳转：在解码过程中，播放器必须有关键帧的信息才能进行解码，因此跳转功能只能通过关键帧跳转。例如，如果关键帧在10s和15s处，要跳转到13s位置，播放器会跳转到10s位置，然后将10s到13s之间的帧全部丢弃，等待3s后继续解码播放。
4. 队列管理：播放器通过管理解码器队列中的帧来实现跳帧。当解码器队列建立完成后，直接丢弃队列中的帧可以有效地减少剩余帧的排队延迟。这种帧跳跃机制，通过控制跳过率来控制延迟改进和帧丢失性能。
5. 帧插值：在倍速播放中，如果简单地增加帧率可能会导致视频的丢帧，影响视频的流畅度。因此，播放器可以采用帧插值技术来生成中间帧，保持视频的流畅度。
6. 音频处理：在视频播放中，音频和视频是需要同步的。当改变视频的播放速度时，音频的播放速度也需要相应地改变。播放器采用时域音高缩放（TDPS）或频域音高缩放（FDPS）技术来保持音频的音高不变。
7. 用户界面控制：播放器提供一个简单易用的用户界面，让用户可以方便地调整播放速度。用户可以通过进度条直接跳转到视频的特定位置，播放器会处理相应的跳帧逻辑。

## 初步方案

针对实时流媒体传输和跳帧处理的问题，存在等待流媒体传输的问题，这会导致处理延迟。可能的解决方案包括

- 多线程处理：使用多线程或多进程来并行处理视频流。一个线程负责从视频流中抓取帧，另一个线程负责处理这些帧。这样可以减少等待时间，因为处理和抓取可以同时进行。
- 缓冲机制：引入一个缓冲区来存储一定数量的帧，这样处理线程可以从缓冲区中获取帧进行处理，而不必等待实时流的传输。这样可以平滑处理过程中的波动，并减少因等待新帧而产生的延迟。
- 使用消息队列：使用消息队列（如RabbitMQ或Kafka）来处理视频流数据，这样可以异步地从队列中读取和处理数据，提高系统的响应性和可扩展性。
- 分布式处理：如果视频流数据量非常大，可以考虑分布式处理，将视频流分割成多个部分，分别在不同的服务器上并行处理。

最后实现一个Demo
- 使用共享队列和多线程
- 其中一个线程负责接收流媒体的数据，设置固定的截断点，将数据放入队列中，存储时就跳帧存储。
- 另一个线程负责从队列中取出数据，进行处理。

# 初步代码实现
## 本地视频逐帧处理

In [1]:
import cv2
import time

video_path = "Pedestrian.mp4"
cap = cv2.VideoCapture(video_path)

# 获取视频的帧率、总帧数和视频时长
fps = cap.get(cv2.CAP_PROP_FPS) 
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
video_duration = total_frames / fps

start_time = time.time()  # 记录开始时间

while cap.isOpened():
    # read()方法返回两个值，第一个是布尔值表示是否读取到帧，第二个是帧本身。这个过程有图像的解码过程，即从缓冲区拿出数据 + 对数据进行解码
    ret, frame = cap.read() 
    if not ret:
        break
    pass

cap.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read all frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Video duration: {video_duration} seconds")

Total time to read all frames: 10.985857009887695 seconds
Video FPS: 24.0
Total frames: 7203
Video duration: 300.125 seconds


## 本地视频跳帧处理

In [2]:
import cv2
import time

video_path = "Pedestrian.mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频的帧率
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 获取视频的总帧数
video_duration = total_frames / fps  # 计算视频时长

start_time = time.time()  # 记录开始时间

interval = 4  # 每隔X帧处理一次
idx = 0

while cap.isOpened():
    idx += 1
    ret = cap.grab()  # 仅仅抓取帧，不解码
    if not ret:
        break
    if idx % interval == 1:
        ret, frame = cap.retrieve()  # 解码并获取帧
        if frame is None:
            break
        pass

cap.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read all frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Video duration: {video_duration} seconds")


Total time to read all frames: 4.50865364074707 seconds
Video FPS: 24.0
Total frames: 7203
Video duration: 300.125 seconds


## 本地视频倍速播放

In [17]:
import cv2
import time

video_path = "Pedestrian.mp4"
output_path = "Pedestrian_fast.mp4"

cap = cv2.VideoCapture(video_path)

# 获取视频的帧率、总帧数和视频时长
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

# 设置倍速因子
speed_factor = 2  # 2倍速播放

# 获取视频帧宽度和高度
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义视频编解码器并创建 VideoWriter 对象
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps * speed_factor, (frame_width, frame_height))

start_time = time.time()  # 记录开始时间

while True:
    ret, frame = cap.read()  # 读取当前帧
    if not ret:
        break
    # 处理帧
    pass
    # 将帧写入输出视频
    out.write(frame)

# 释放视频资源并记录结束时间
cap.release()
out.release()
end_time = time.time()

total_time = end_time - start_time

print(f"Total time to read all frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Video duration: {video_duration} seconds")

Total time to read all frames: 32.049954891204834 seconds
Video FPS: 24.0
Total frames: 7203
Video duration: 300.125 seconds


## 在线视频逐帧处理

In [3]:
import cv2
import time

video_path = "https://img.qunliao.info/4oEGX68t_9505974551.mp4"
cap = cv2.VideoCapture(video_path)

# 获取视频的帧率、总帧数和视频时长
fps = cap.get(cv2.CAP_PROP_FPS) 
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
video_duration = total_frames / fps

start_time = time.time()  # 记录开始时间

while cap.isOpened():
    # read()方法返回两个值，第一个是布尔值表示是否读取到帧，第二个是帧本身。这个过程有图像的解码过程，即从缓冲区拿出数据 + 对数据进行解码
    ret, frame = cap.read() 
    if not ret:
        break
    pass

cap.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read all frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Video duration: {video_duration} seconds")

Total time to read all frames: 4.6286749839782715 seconds
Video FPS: 25.0
Total frames: 7047
Video duration: 281.88 seconds


## 在线视频跳帧处理

In [4]:
import cv2
import time

video_path = "https://img.qunliao.info/4oEGX68t_9505974551.mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频的帧率
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 获取视频的总帧数
video_duration = total_frames / fps  # 计算视频时长

start_time = time.time()  # 记录开始时间

interval = 4  # 每隔X帧处理一次
idx = 0

while cap.isOpened():
    idx += 1
    ret = cap.grab()  # 仅仅抓取帧，不解码
    if not ret:
        break
    if idx % interval == 1:
        ret, frame = cap.retrieve()  # 解码并获取帧
        if frame is None:
            break
        pass

cap.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read all frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Video duration: {video_duration} seconds")


Total time to read all frames: 2.4956533908843994 seconds
Video FPS: 25.0
Total frames: 7047
Video duration: 281.88 seconds


## 在线视频倍速播放

In [20]:
import cv2
import time

video_path = "https://img.qunliao.info/4oEGX68t_9505974551.mp4"
output_path = "Soccer.mp4"

cap = cv2.VideoCapture(video_path)

# 获取视频的帧率、总帧数和视频时长
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps

# 设置倍速因子
speed_factor = 2  # 2倍速播放

# 获取视频帧宽度和高度
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义视频编解码器并创建 VideoWriter 对象
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps * speed_factor, (frame_width, frame_height))

start_time = time.time()  # 记录开始时间

while True:
    ret, frame = cap.read()  # 读取当前帧
    if not ret:
        break
    # 处理帧
    pass
    # 将帧写入输出视频
    out.write(frame)

# 释放视频资源并记录结束时间
cap.release()
out.release()
end_time = time.time()

total_time = end_time - start_time

print(f"Total time to read all frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames: {total_frames}")
print(f"Video duration: {video_duration} seconds")

Total time to read all frames: 16.089090824127197 seconds
Video FPS: 25.0
Total frames: 7047
Video duration: 281.88 seconds


## 实时流媒体逐帧处理

In [ ]:
import cv2

# RTMP 流地址
rtmp_url = "rtmp://liteavapp.qcloud.com/live/liteavdemoplayerstreamid"

# 打开视频流
cap = cv2.VideoCapture(rtmp_url)

if not cap.isOpened():
    print("无法打开视频流")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("无法读取视频帧")
        break

    # 显示视频帧
    cv2.imshow('RTMP Stream', frame)

    # 按 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放视频捕获对象并关闭所有 OpenCV 窗口
cap.release()
cv2.destroyAllWindows()

In [5]:
import cv2
import time

video_path = "rtmp://liteavapp.qcloud.com/live/liteavdemoplayerstreamid"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS) # 获取视频的帧率
max_frames = 1000  # 读取多少帧
frame_count = 0  # 初始化帧计数器
start_time = time.time()  # 记录开始时间

while cap.isOpened():
    ret, frame = cap.read() 
    if not ret:
        break
    
    frame_count += 1  
    
    # 检查是否达到最大秒数
    if frame_count >= max_frames:
        break

cap.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames read: {frame_count}")

Total time to read frames: 26.93897271156311 seconds
Video FPS: 30.0
Total frames read: 1000


## 实时流媒体跳帧处理

In [6]:
import cv2
import time

video_path = "rtmp://liteavapp.qcloud.com/live/liteavdemoplayerstreamid"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频的帧率

interval = 4  # 每隔X帧处理一次
idx = 0

max_frames = 1000  # 读取多少帧
all_frame_count = 0
truncated_frame_count = 0

start_time = time.time()  # 记录开始时间

while cap.isOpened():
    idx += 1
    ret = cap.grab()  # 仅仅抓取帧，不解码
    all_frame_count += 1
    if not ret:
        break
    if idx % interval == 1:
        ret, frame = cap.retrieve()  # 解码并获取帧
        if frame is None:
            break
        truncated_frame_count += 1
        

    current_time = time.time()
    
    # 检查是否达到最大帧数
    if all_frame_count >= max_frames:
        break

cap.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames read: {all_frame_count}")
print(f"Total frames processed: {truncated_frame_count}")

Total time to read frames: 24.243430376052856 seconds
Video FPS: 30.0
Total frames read: 1000
Total frames processed: 250


## 实时流媒体倍速播放

In [23]:
import cv2
import time

video_path = "rtmp://liteavapp.qcloud.com/live/liteavdemoplayerstreamid"
output_path = "Stream.mp4"

cap = cv2.VideoCapture(video_path)

# 获取视频的帧率和帧尺寸
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 设置倍速因子
speed_factor = 2 

# 设置读取的最大帧数
max_frames = 1000  

frame_count = 0
processed_frame_count = 0

# 初始化视频写入器
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

start_time = time.time()  # 记录开始时间

while cap.isOpened() and frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # 仅处理每 speed_factor 帧中的一帧
    if frame_count % speed_factor == 0:
        out.write(frame)
        processed_frame_count += 1

cap.release()
out.release()
end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames read: {frame_count}")
print(f"Total frames processed: {processed_frame_count}")

Total time to read frames: 26.056715965270996 seconds
Video FPS: 30.0
Total frames read: 1000
Total frames processed: 500


## 实时流媒体跳帧处理: 队列+多线程Demo

In [7]:
import cv2
import time
import threading
import queue

video_path = "rtmp://liteavapp.qcloud.com/live/liteavdemoplayerstreamid"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频的帧率

interval = 4  # 每隔X帧处理一次
max_frames = 1000  # 读取多少帧

frame_queue = queue.Queue(maxsize=10)  # 帧队列，最大存储10帧
stop_event = threading.Event()  # 停止事件

def read_frames():
    idx = 0
    all_frame_count = 0
    while not stop_event.is_set() and cap.isOpened():
        ret = cap.grab()  # 仅仅抓取帧，不解码
        if not ret:
            break
        idx += 1
        all_frame_count += 1
        if idx % interval == 1:
            frame_queue.put((idx, cap.retrieve()))
        # 检查是否达到最大帧数
        if all_frame_count >= max_frames:
            break
    cap.release()
    stop_event.set()  # 设置停止事件

# 启动读取帧的线程
thread = threading.Thread(target=read_frames)
thread.start()

start_time = time.time()  # 记录开始时间
truncated_frame_count = 0

try:
    while not stop_event.is_set() or not frame_queue.empty():
        try:
            all_frame_count, frame = frame_queue.get(timeout=0.1)
            truncated_frame_count += 1
            # 在这里处理帧
            pass
        except queue.Empty:
            continue
finally:
    stop_event.set()  # 确保停止事件被设置
    thread.join()  # 等待读取线程结束

end_time = time.time()  # 记录结束时间
total_time = end_time - start_time  # 计算总时间

print(f"Total time to read frames: {total_time} seconds")
print(f"Video FPS: {fps}")
print(f"Total frames read: {all_frame_count}")
print(f"Total frames processed: {truncated_frame_count}")


Total time to read frames: 21.248888969421387 seconds
Video FPS: 30.0
Total frames read: 997
Total frames processed: 250
